## [Speed Up Your Python Program With Concurrency](https://realpython.com/python-concurrency/)

#### What Is Concurrency?
- Concurrency: simultaneous occurrence.
- Think of "thread, task, and process" as different trains of thought. Each one can be stopped at certain points, and the CPU that is processing them can switch to a different one. The state of each one is saved so it can be restarted right where it was interrupted.
- Only `multiprocessing` actually runs these trains of thought at literally the same time. `Threading` and `asyncio` both run on a single processor and therefore only run one at a time. They just cleverly find ways to take turns to speed up the overall process.
- The way the threads or tasks take turns is the big difference between `threading` and `asyncio`. 
    - In `threading`, the operating system actually knows about each thread and can interrupt it at any time to start running a different thread. This is called [**pre-emptive multitasking**](https://en.wikipedia.org/wiki/Preemption_%28computing%29#Preemptive_multitasking) since the operating system can pre-empt your thread to make the switch.
    - `Asyncio` uses [**cooperative multitasking**](https://en.wikipedia.org/wiki/Cooperative_multitasking). The tasks must cooperate by announcing when they are ready to be switched out. That means that the code in the task has to change slightly to make this happen. The benefit of doing this extra work up front is that you always know where your task will be swapped out.

#### What Is Parallelism?
- With `multiprocessing`, Python creates new processes. A `process` here can be thought of as almost a completely different program, though technically they’re usually defined as a collection of resources where the resources include memory, file handles and things like that. One way to think about it is that each process runs in its own Python interpreter.

#### When Is Concurrency Useful?
- Concurrency can make a big difference for two types of problems. These are generally called `CPU-bound` and `I/O-bound`.
- `I/O-bound problems` cause your program to slow down because it frequently must wait for input/output (I/O) from some external resource. They arise frequently when your program is working with things that are much slower than your CPU. The slow things your program will interact with most frequently are the file system and network connections.
- On the flip side, there are classes of programs that do significant computation without talking to the network or accessing a file. These are the `CPU-bound programs`, because the resource limiting the speed of your program is the CPU, not the network or the file system.
- Adding concurrency to your program adds extra code and complications, so you’ll need to decide if the potential speed up is worth the extra effort.

#### How to Speed Up an I/O-Bound Program
##### Synchronous Version
- Read the `io_non_concurrency.py` file.
- Advantages: easy to write and debug; more straight-forward to think about; one train of thought running, so you can predict what the next step is and how it will behave.
- Problem: it’s relatively slow.

##### threading Version
- Read the `io_threading.py` file simultaneously while reading what's written below.
- `ThreadPoolExecutor = Thread + Pool + Executor`
    - `Thread`: train of thought.
    - `Pool`: This object is going to create a pool of threads, each of which can run concurrently.
    - `Executor` is the part that’s going to control how and when each of the threads in the pool will run. It will execute the request in the pool.
- The standard library implements ThreadPoolExecutor as a context manager so you can use the with syntax to manage creating and freeing the pool of Threads.
- `.map()`: This method runs the passed-in function on each of the sites in the list. It automatically runs them concurrently using the pool of threads it is managing.
- Starting with Python 3.2, the standard library added a higher-level abstraction called `Executors` that manage many of the details for you if you don’t need that fine-grained control. (details like like `Thread.start()`, `Thread.join()`, and `Queue`.)
- In our example each thread needs to create its own `requests.Session()` object. This is one of the interesting and difficult issues with threading. Because the operating system is in control of when your task gets interrupted and another task starts, any data that is shared between the threads needs to be protected, or **thread-safe**. Unfortunately `requests.Session()` is not thread-safe.
- There are several strategies for making data accesses thread-safe depending on what the data is and how you’re using it. One of them is to use thread-safe data structures like `Queue` from Python’s `queue` module.
- These objects use low-level primitives like `threading.Lock` to ensure that only one thread can access a block of code or a bit of memory at the same time. You are using this strategy indirectly by way of the ThreadPoolExecutor object. 
- Another strategy to use here is something called *thread local storage*. `threading.local()` creates an object that looks like a global but is specific to each individual thread. The object itself takes care of separating accesses from different threads to different data.
- When get_session() is called, the session it looks up is specific to the particular thread on which it’s running. So each thread will create a single session the first time it calls get_session() and then will simply use that session on each subsequent call throughout its lifetime.
- Number of threads: the correct number of threads is not a constant from one task to another. Some experimentation is required. However, note that if the number of threads increase then the extra overhead of creating and destroying the threads erases any time savings.
- Advantage: It's fast.
- Problem:
    - it takes a little more code, and you really have to give some thought to what data is shared between threads.
    - Threads can interact in ways that are subtle and hard to detect. These interactions can cause *race conditions* that frequently result in random, intermittent bugs that can be quite difficult to find.
- **Race conditions** are an entire class of subtle bugs that can and frequently do happen in multi-threaded code. Race conditions happen because the programmer has not sufficiently protected data accesses to prevent threads from interfering with each other. What’s going on here is that the operating system is controlling when your thread runs and when it gets swapped out to let another thread run. This thread swapping can occur at any point, even while doing sub-steps of a Python statement.

##### asyncio Version